In [ ]:
import imageio
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from starter.utils import get_device, get_mesh_renderer, load_cow_mesh, get_points_renderer, unproject_depth_image
from starter.render_generic import load_rgbd_data
import mcubes
import pytorch3d
import torch
from tqdm import tqdm
from PIL import Image, ImageDraw

device = get_device()
cow_path = "data/cow.obj"
image_size=256

## Helper functions

In [ ]:
def create_gif_from_image_list(images_list: list[np.ndarray], gif_path: Path, FPS=15):
    # images_list is a list of (H,W,3) images
    assert images_list[0].shape[2] == 3
    
    frame_duration_ms = 1000 // FPS
    imageio.mimsave(gif_path, images_list, duration=frame_duration_ms, loop=0)

## Question 1: Practicing with Cameras

### 1.1: 360-deg renders

In [ ]:



# Get the renderer.
renderer = get_mesh_renderer(image_size=image_size)

# Get the vertices, faces, and textures.
vertices, faces = load_cow_mesh(cow_path)
vertices = vertices.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
faces = faces.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)
textures = torch.ones_like(vertices)  # (1, N_v, 3)
textures = textures * torch.tensor([0.7, 0.7, 1])  # (1, N_v, 3)

mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)
mesh = mesh.to(device)


# Place a point light in front of the cow.
lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

images_list = []

for i in tqdm(range(0,360,10), desc="Rendering cow..."):

    theta = np.radians(i)
    c, s = np.cos(theta), np.sin(theta)
    R = torch.tensor([[c, 0, s], [0, 1, 0], [-s, 0, c]]).unsqueeze(0)

    # Prepare the camera:
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R, T=torch.tensor([[0, 0, 3]]), fov=60, device=device
    )
    
    rend = renderer(mesh, cameras=cameras, lights=lights)
    img = rend.cpu().numpy()[0, ..., :3]
        
    img *= 255
    img = img.astype('uint8')
    images_list.append(img)

create_gif_from_image_list(images_list, Path('out.gif'))

Rendering cow...: 100%|██████████| 36/36 [00:08<00:00,  4.08it/s]


## HW1Q1 result
<img src="out.gif" width="256" align="center">

## 1.2: Dolly Zoom

In [ ]:

num_frames=10
duration=3
output_file="output/dolly.gif"

mesh = pytorch3d.io.load_objs_as_meshes(["data/cow_on_plane.obj"])
mesh = mesh.to(device)
renderer = get_mesh_renderer(image_size=image_size, device=device)
lights = pytorch3d.renderer.PointLights(location=[[0.0, 0.0, -3.0]], device=device)

fovs = torch.linspace(5, 120, num_frames)

renders = []

width = 5
for fov in tqdm(fovs):
    # distance = 50
    distance = width / (2 * np.tan(0.5 * np.radians(fov))) # TODO: change this.
    T = [[0, 0, distance]]  # TODO: Change this.
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(fov=fov, T=T, device=device)
    rend = renderer(mesh, cameras=cameras, lights=lights)
    rend = rend[0, ..., :3].cpu().numpy()  # (N, H, W, 3)
    renders.append(rend)

images = []
for i, r in enumerate(renders):
    image = Image.fromarray((r * 255).astype(np.uint8))
    draw = ImageDraw.Draw(image)
    draw.text((20, 20), f"fov: {fovs[i]:.2f}", fill=(255, 0, 0))
    images.append(np.array(image))

create_gif_from_image_list(images, Path('hw1q1p2.gif'))


100%|██████████| 10/10 [00:02<00:00,  3.97it/s]


## q1.2 Dolly Zoom Result
<img src="hw1q1p2.gif" width="256">

# Question 2: Practicing with Meshes

## Helper Function

In [ ]:
def render_mesh_to_gif(gif_path:Path, desc: str, V: torch.tensor, F: torch.tensor):    
    # Get the renderer.
    renderer = get_mesh_renderer(image_size=image_size)

    # Get the vertices, faces, and textures.
    vertices = V.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
    faces = F.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)
    textures = torch.ones_like(vertices)  # (1, N_v, 3)
    textures = textures * torch.tensor([0.7, 0.7, 1])  # (1, N_v, 3)
    mesh = pytorch3d.structures.Meshes(
        verts=vertices,
        faces=faces,
        textures=pytorch3d.renderer.TexturesVertex(textures),
    )
    mesh = mesh.to(device)
    
    # Place a point light in front of the cow.
    lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

    images_list = []

    for i in tqdm(range(0, 360, 10), desc="Rendering " + desc + "..."):
        theta = np.radians(i)
        c, s = np.cos(theta), np.sin(theta)
        R = torch.tensor([[c, 0, s], [0, 1, 0], [-s, 0, c]]).unsqueeze(0)

        # Prepare the camera:
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(
            R=R, T=torch.tensor([[0, 0, 3]]), fov=60, device=device
        )
        
        rend = renderer(mesh, cameras=cameras, lights=lights)
        img = rend.cpu().numpy()[0, ..., :3]
            
        img *= 255
        img = img.astype('uint8')
        images_list.append(img)
    
    create_gif_from_image_list(images_list, gif_path)


## Q2.1 Construct a Tetrahedron

In [ ]:
V_tetra = torch.tensor([
    [-0.5,-0.5,-0.5],
    [0,1,0],
    [0,0,1],
    [1,0,0]
], dtype=torch.float32)

F_tetra = torch.tensor([
    [0,1,2],
    [1,2,3],
    [0,2,3],
    [0,1,3]
], dtype=torch.long)
render_mesh_to_gif("hw1q2_tetrahedron.gif", "tetrahedron", V=V_tetra, F=F_tetra)

Rendering tetrahedron...: 100%|██████████| 36/36 [00:00<00:00, 100.15it/s]


## Q2.1 Tetrahedron
<image src="hw1q2_tetrahedron.gif" width="256">

## Q2.2 Construct a Cube

In [ ]:

V_cube = torch.tensor([
    [-0.5,-0.5,-0.5],
    [0.5,-0.5,-0.5],
    [-0.5,0.5,-0.5],
    [0.5,0.5,-0.5],
    
    [-0.5,-0.5,0.5],
    [0.5,-0.5,0.5],
    [-0.5,0.5,0.5],
    [0.5,0.5,0.5]
], dtype=torch.float32)

F_cube = torch.tensor([
    [0,1,2],
    [1,2,3],
    
    [4,5,6],
    [5,6,7],
    
    [0,1,4],
    [1,4,5],
    
    [2,3,6],
    [3,6,7],
    
    [0,2,4],
    [2,4,6],
    
    [1,3,5],
    [3,5,7]
    
], dtype=torch.long)
render_mesh_to_gif("hw1q2_cube.gif", "cube", V=V_cube, F=F_cube)

Rendering cube...: 100%|██████████| 36/36 [00:00<00:00, 94.80it/s]


## Q2.2 Cube
<image src="hw1q2_cube.gif" width="256">

# Q3. Retexturing a mesh

In [ ]:
# Get the renderer.
renderer = get_mesh_renderer(image_size=image_size)

# Get the vertices, faces, and textures.
vertices_cow, faces_cow = load_cow_mesh(cow_path)
vertices = vertices_cow.unsqueeze(0)  # (N_v, 3) -> (1, N_v, 3)
faces = faces_cow.unsqueeze(0)  # (N_f, 3) -> (1, N_f, 3)
zs = vertices[0,:,2]
z_max = torch.max(zs)
z_min = torch.min(zs)
color1 =  torch.tensor([[0,0,1]], dtype=torch.float32)
color2 = torch.tensor([[1,0,0]], dtype=torch.float32)
alphas = ((zs - z_min) / (z_max - z_min)).unsqueeze(1)
textures = (alphas @ color2 + (1 - alphas) @ color1).unsqueeze(0) # (1, N_v, 3)

mesh = pytorch3d.structures.Meshes(
    verts=vertices,
    faces=faces,
    textures=pytorch3d.renderer.TexturesVertex(textures),
)
mesh = mesh.to(device)


# Place a point light in front of the cow.
lights = pytorch3d.renderer.PointLights(location=[[0, 0, -3]], device=device)

images_list = []

for i in tqdm(range(0,360,10), desc="Rendering cow..."):

    theta = np.radians(i)
    c, s = np.cos(theta), np.sin(theta)
    R = torch.tensor([[c, 0, s], [0, 1, 0], [-s, 0, c]]).unsqueeze(0)

    # Prepare the camera:
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R, T=torch.tensor([[0, 0, 3]]), fov=60, device=device
    )
    
    rend = renderer(mesh, cameras=cameras, lights=lights)
    img = rend.cpu().numpy()[0, ..., :3]
        
    img *= 255
    img = img.astype('uint8')
    images_list.append(img)

create_gif_from_image_list(images_list, Path('hw1q3_color.gif'))

Rendering cow...: 100%|██████████| 36/36 [00:00<00:00, 162.10it/s]


## Q3 Results
<image src="hw1q3_color.gif" width="256">

# Q4. Camera Transformations

In [8]:
def render_textured_cow(
    cow_path="data/cow.obj",
    R_relative=[[1, 0, 0], [0, 1, 0], [0, 0, 1]],
    T_relative=[0, 0, 0],
):
    meshes = pytorch3d.io.load_objs_as_meshes([cow_path]).to(device)
    R_relative = torch.tensor(R_relative).float()
    T_relative = torch.tensor(T_relative).float()
    R = R_relative @ torch.tensor([[1.0, 0, 0], [0, 1, 0], [0, 0, 1]])
    T = R_relative @ torch.tensor([0.0, 0, 3]) + T_relative
    renderer = get_mesh_renderer(image_size=256)
    cameras = pytorch3d.renderer.FoVPerspectiveCameras(
        R=R.unsqueeze(0), T=T.unsqueeze(0), device=device,
    )
    lights = pytorch3d.renderer.PointLights(location=[[0, 0.0, -3.0]], device=device,)
    rend = renderer(meshes, cameras=cameras, lights=lights)
    return rend[0, ..., :3].cpu().numpy()

Rs = [
        [[1,0,0],[0,1,0],[0,0,1]], #identity
        [[0,1,0],[-1,0,0],[0,0,1]],#RotZ_cw_90
        [[1,0,0],[0,1,0],[0,0,1]],
        [[1,0,0],[0,1,0],[0,0,1]], 
        [[0,0,1],[0,1,0],[-1,0,0]],#RotY_cw_90
    ]
    
Ts = [
    [0,0,0],
    [0,0,0],
    [0,0,2], #zoom out
    [0.5,-0.5,0], #move bottom left
    [-3,0,3], #reset cam after rotation and go to z=+3
]

jpg_names = [
    "identity",
    "q1_rotz_cw_90",
    "q2_zoom_out",
    "q3_move_bottom_left",
    "q4_roty_cw_90",
]

for i in range(len(Rs)):
    img = render_textured_cow(R_relative=Rs[i], T_relative=Ts[i])
    plt.imsave("hw1q4_"+jpg_names[i]+".jpg", img)

# TODO add more text here

# Q5 Rendering Generic 3D Representations
## Helper Functions

In [ ]:

def render_pointcloud_to_gif(
    V: torch.Tensor,
    rgb: torch.Tensor,
    gif_path: Path,
    background_color=(1, 1, 1),
    downsample_factor=1,
):
    """
    Renders a point cloud.
    """
    renderer = get_points_renderer(
        image_size=image_size, background_color=background_color
    )
    
    verts = V[::downsample_factor].to(device).unsqueeze(0)
    rgb = rgb[::downsample_factor].to(device).unsqueeze(0)
    point_cloud = pytorch3d.structures.Pointclouds(points=verts, features=rgb)
    
    image_list = []
    for azimuth in tqdm(range(0, 360, 10), desc="Rendering pointcloud..."): 
        R, T = pytorch3d.renderer.look_at_view_transform(6, 10, azimuth)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)
        rend = renderer(point_cloud, cameras=cameras)
        img = rend.cpu().numpy()[0, ..., :3]  # (B, H, W, 4) -> (H, W, 3)
        img *= 255
        img = img.astype('uint8')
        image_list.append(img)
        
    create_gif_from_image_list(image_list, gif_path)

## Q5.1: Rendering PointCloud from RGBD Images

In [ ]:
data = load_rgbd_data() #dict_keys(['rgb1', 'mask1', 'depth1', 'rgb2', 'mask2', 'depth2', 'cameras1', 'cameras2'])
pc1_points, pc1_rgb = unproject_depth_image(image=torch.Tensor(data['rgb1']), mask=torch.Tensor(data['mask1']), depth=torch.Tensor(data['depth1']), camera=data['cameras1'])
pc2_points, pc2_rgb = unproject_depth_image(image=torch.Tensor(data['rgb2']), mask=torch.Tensor(data['mask2']), depth=torch.Tensor(data['depth2']), camera=data['cameras2'])
union_points = torch.vstack([pc1_points,pc2_points])
union_rgb = torch.vstack([pc1_rgb, pc2_rgb])

render_pointcloud_to_gif(V=pc1_points, rgb=pc1_rgb, gif_path="hw1q5p1_pc1.gif", downsample_factor=10)
render_pointcloud_to_gif(V=pc2_points, rgb=pc2_rgb, gif_path="hw1q5p1_pc2.gif", downsample_factor=10)
render_pointcloud_to_gif(V=union_points, rgb=union_rgb, gif_path="hw1q5p1_pc_both.gif", downsample_factor=10)

/Users/tomg/miniconda3/envs/l3d/lib/python3.10/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:4324.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Rendering pointcloud...:  81%|████████  | 29/36 [01:07<00:16,  2.33s/it]


KeyboardInterrupt: 

### Q5.1 First Image PC | Second Image PC | Union PC
<image src="hw1q5p1_pc1.gif" width="256">
<image src="hw1q5p1_pc2.gif" width="256">
<image src="hw1q5p1_pc_both.gif" width="256">

## Q5.2 Parametric Functions

In [ ]:

def sample_torus_to_pointcloud(num_samples=200):
    phi = torch.linspace(0, 2 * np.pi, num_samples)
    theta = torch.linspace(0, np.pi, num_samples)
    # Densely sample phi and theta on a grid
    Phi, Theta = torch.meshgrid(phi, theta)
    
    R = 2
    r = 0.5
    x = (R + r * torch.sin(Theta)) * torch.cos(Phi)
    y = (R + r * torch.sin(Theta)) * torch.sin(Phi)
    z = r * torch.cos(Theta)

    points = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)
    color = (points - points.min()) / (points.max() - points.min())
    
    return points, color

def sample_cyl_to_pointcloud(num_samples=200):
    u = torch.linspace(0, np.pi, num_samples)
    v = torch.linspace(0, 2*np.pi, num_samples)

    u,v = torch.meshgrid(u, v)
    r = 2
    
    x = r * np.cos(v)
    y = r * np.sin(v)
    z = u

    points = torch.stack((x.flatten(), y.flatten(), z.flatten()), dim=1)
    color = (points - points.min()) / (points.max() - points.min())
    
    return points, color

torus_pts, torus_color = sample_torus_to_pointcloud(num_samples=200)

render_pointcloud_to_gif(V=torus_pts, rgb=torus_color, gif_path="hw1q5p2_torus.gif", downsample_factor=10)

cyl_pts, cyl_color = sample_cyl_to_pointcloud(num_samples=200)

render_pointcloud_to_gif(V=cyl_pts, rgb=cyl_color, gif_path="hw1q5p2_cyl.gif", downsample_factor=1)

Rendering pointcloud...: 100%|██████████| 36/36 [00:00<00:00, 259.41it/s]
/tmp/ipykernel_12044/1847027474.py:32: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  x = r * np.cos(v)
/tmp/ipykernel_12044/1847027474.py:33: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  y = r * np.sin(v)
Rendering pointcloud...: 100%|██████████| 36/36 [00:00<00:00, 269.21it/s]


## Q5.2 Results: Torus | Cylinder
<image src="hw1q5p2_torus.gif" width="256">
<image src="hw1q5p2_cyl.gif" width="256">

## Q5.3 Implicit Surfaces

In [ ]:
def render_mesh_to_gif(
        mesh: pytorch3d.structures.Meshes,
        gif_path: Path):
    lights = pytorch3d.renderer.PointLights(location=[[0, 0.0, -4.0]], device=device)
    renderer = get_mesh_renderer(image_size=image_size, device=device)

    image_list = []
    for azimuth in tqdm(range(0, 360, 10), desc="Rendering mesh..."): 
        R, T = pytorch3d.renderer.look_at_view_transform(6, 10, azimuth)
        cameras = pytorch3d.renderer.FoVPerspectiveCameras(R=R, T=T, device=device)
        rend = renderer(mesh, cameras=cameras, lights=lights)
        img = rend.cpu().numpy()[0, ..., :3].clip(0,1)  # (B, H, W, 4) -> (H, W, 3)
        img *= 255
        img = img.astype('uint8')
        image_list.append(img)
        
    create_gif_from_image_list(image_list, gif_path)


In [ ]:

def create_torus_mesh(voxel_size=64):
    min_value = -1.6
    max_value = 1.6
    R = 1
    r = 0.5
    X, Y, Z = torch.meshgrid([torch.linspace(min_value, max_value, voxel_size)] * 3)
    voxels = (torch.sqrt(X**2 + Y**2) - R)**2 + Z**2 - r**2
    vertices, faces = mcubes.marching_cubes(mcubes.smooth(voxels), isovalue=0)
    vertices = torch.tensor(vertices).float()
    faces = torch.tensor(faces.astype(int))
    # Vertex coordinates are indexed by array position, so we need to
    # renormalize the coordinate system.
    vertices = (vertices / voxel_size) * (max_value - min_value) + min_value
    textures = (vertices - vertices.min()) / (vertices.max() - vertices.min())
    textures = pytorch3d.renderer.TexturesVertex(vertices.unsqueeze(0))

    mesh = pytorch3d.structures.Meshes([vertices], [faces], textures=textures).to(
        device
    )
    return mesh


def create_whatever_mesh(voxel_size=64):
    min_value = -5
    max_value = 5
    X, Y, Z = torch.meshgrid([torch.linspace(min_value, max_value, voxel_size)] * 3)
    voxels = X**4 + Y**4 + Z**4 + 2 * X**2 * Y**2 + 2 * X**2 * Z**2 + 2 * Y**2 * Z**2 + 8*X*Y*Z - 8 * X**2 - 8 * Y**2 - 8 * Z**2 + 15
    vertices, faces = mcubes.marching_cubes(mcubes.smooth(voxels), isovalue=0)
    vertices = torch.tensor(vertices).float()
    faces = torch.tensor(faces.astype(int))
    # Vertex coordinates are indexed by array position, so we need to
    # renormalize the coordinate system.
    vertices = (vertices / voxel_size) * (max_value - min_value) + min_value
    textures = (vertices - vertices.min()) / (vertices.max() - vertices.min())
    textures = pytorch3d.renderer.TexturesVertex(vertices.unsqueeze(0))

    mesh = pytorch3d.structures.Meshes([vertices], [faces], textures=textures).to(device)
    return mesh

mesh = create_torus_mesh()
render_mesh_to_gif(mesh, "hw1q5p3_torus.gif")

mesh2 = create_whatever_mesh()
render_mesh_to_gif(mesh2, "hw1q5p3_whatever.gif")

Rendering mesh...: 100%|██████████| 36/36 [00:00<00:00, 171.45it/s]


## Q5.3 Implicit Surfaces: Torus | Custom Object
<image src="hw1q5p3_torus.gif" width="256">
<image src="hw1q5p3_whatever.gif" width="256">

# TODO may have to change custom object if they are iffy about it

# Q6: Do Something Fun

# TODO have fun